In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications import Xception
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
##############
# parameters #
##############
# dontFreezeLast = 0;

# patience = 10;

# gpuName = '/device:GPU:0'
# workers = 2;
# histogram_freq = 0;

# epochs = 100;
# validation_size=0.3;

modelPath = '../models/Xception_opt/';
modelName = 'run1.h5';
os.environ["CUDA_VISIBLE_DEVICES"]="0"

if not os.path.exists(modelPath):
    os.makedirs(modelPath)

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
# df_train = pd.read_csv('../input/labels.csv')
# df_test = pd.read_csv('../input/sample_submission.csv')

# print('Training images: ',df_train.shape[0])
# print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

In [4]:
#df_train.head(10)

We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
# targets_series = pd.Series(df_train['breed'])
# one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
# one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
# im_size = 90

In [8]:
# x_train = []
# y_train = []
# x_test = []

In [9]:
# i = 0 
# for f, breed in tqdm(df_train.values[:10]):
#     img = cv2.imread('../input/train/{}.jpg'.format(f))
#     label = one_hot_labels[i]
#     x_train.append(cv2.resize(img, (im_size, im_size)))
#     y_train.append(label)
#     i += 1

In [10]:
# for f in tqdm(df_test['id'].values):
#     img = cv2.imread('../input/test/{}.jpg'.format(f))
#     x_test.append(cv2.resize(img, (im_size, im_size)))

In [11]:
# y_train_raw = np.array(y_train, np.uint8)
# x_train_raw = np.array(x_train, np.float32) / 255.
# x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
# print(x_train_raw.shape)
# print(y_train_raw.shape)
# print(x_test.shape)

We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
# num_class = y_train_raw.shape[1]
# print('Number of classes: ', num_class)

It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [22]:
def data():
    print('Getting data')
    df_train = pd.read_csv('../input/labels.csv')
    df_test = pd.read_csv('../input/sample_submission.csv')
    
    targets_series = pd.Series(df_train['breed'])
    one_hot = pd.get_dummies(targets_series, sparse = True)
    one_hot_labels = np.asarray(one_hot)
    
    im_size = 90
    x_train = []
    y_train = []
    x_test = []
    
    i = 0 
    for f, breed in tqdm(df_train.values[:8000]):
        img = cv2.imread('../input/train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
    
    y_train_raw = np.array(y_train, np.uint8)
    x_train_raw = np.array(x_train, np.float32) / 255.
    num_class = y_train_raw.shape[1]
    
    print('Splitting into training/validation')
    X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
    
    return X_train, Y_train, X_valid, Y_valid

In [16]:
# prepare data and model for hyperas

def model(X_train,Y_train,X_valid,Y_valid):
    print('Creating model')
    base_model = Xception(weights = 'imagenet',
                       include_top=False,
                       input_shape=(im_size, im_size, 3))

    dropout = {{uniform(0.5,1)}};
    layers = {{choice([0,1,2])}};
    dontFreeze = {{choice(list(range(5+1)))}};
    batchSize = {{choice([16,64,256])}};
    
    print()
    print('dropout=',dropout)
    print('layers=',layers)
    print('dontFreeze=',dontFreeze)
    print('batchSize=',batchSize)
    print()
    
    stepsPerEpoch = round( len(X_train)/batchSize );
    
    # Add a new top layer
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    if layers>=2:
        x = Dense(1024,activation='relu')(x)
    if layers>=1:
        x = Dense(512,activation='relu')(x)
    # in any case:
    predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    for i in range(len(base_model.layers)-dontFreeze):
        base_model.layers[i].trainable = False

    # predetermined optimizer
    lr=0.00020389590556056983;
    beta_1=0.9453158868247398;
    beta_2=0.9925872692991417;
    decay=0.000821336141287018;
    adam = keras.optimizers.Adam(lr=lr,beta_1=beta_1,beta_2=beta_2,decay=decay)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=adam, 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=1));


    # data augmentation & fitting
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.5,
        zoom_range=0.5,
        horizontal_flip=True,
        vertical_flip=True);
    
    model.fit_generator(
        datagen.flow(X_train,Y_train,batch_size=batchSize),
        steps_per_epoch=stepsPerEpoch,
        epochs=150,
        verbose=1,
        validation_data=(X_valid,Y_valid),
        workers=2,
        shuffle=True,
        callbacks=callbacks_list)
#     model.fit(X_train, Y_train,
#       epochs=100,
#       batch_size = batchSize,
#       validation_data=(X_valid, Y_valid),
#       verbose=1,
#       callbacks=callbacks_list)

    score, acc = model.evaluate(X_valid, Y_valid, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


In [17]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name='XceptionOptimization_1')

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.applications import Xception
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import random
except:
    pass

try:
    import os
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import cv2
except:
    pa

100%|██████████| 10222/10222 [00:29<00:00, 344.02it/s]


Splitting into training/validation
Creating model

dropout= 0.6064002165637792
layers= 2
dontFreeze= 3
batchSize= 64

Epoch 1/150
112/112 [==============================] - 35s 316ms/step - loss: 4.8225 - acc: 0.0126 - val_loss: 4.7719 - val_acc: 0.0163
Epoch 2/150
112/112 [==============================] - 24s 217ms/step - loss: 4.7440 - acc: 0.0228 - val_loss: 4.6149 - val_acc: 0.0496
Epoch 3/150
112/112 [==============================] - 24s 217ms/step - loss: 4.6401 - acc: 0.0364 - val_loss: 4.4221 - val_acc: 0.0623
Epoch 4/150
112/112 [==============================] - 24s 217ms/step - loss: 4.5119 - acc: 0.0460 - val_loss: 4.2803 - val_acc: 0.0747
Epoch 5/150
112/112 [==============================] - 24s 215ms/step - loss: 4.3910 - acc: 0.0672 - val_loss: 4.1704 - val_acc: 0.0841
Epoch 6/150
112/112 [==============================] - 24s 214ms/step - loss: 4.2991 - acc: 0.0710 - val_loss: 4.1244 - val_acc: 0.0877
Epoch 7/150
112/112 [==============================] - 24s 214ms/s

28/28 [==============================] - 19s 678ms/step - loss: 4.2225 - acc: 0.0870 - val_loss: 3.8909 - val_acc: 0.1249
Epoch 12/150
28/28 [==============================] - 19s 679ms/step - loss: 4.1965 - acc: 0.0984 - val_loss: 3.8830 - val_acc: 0.1278
Epoch 13/150
28/28 [==============================] - 19s 677ms/step - loss: 4.1349 - acc: 0.1009 - val_loss: 3.8901 - val_acc: 0.1288
Epoch 14/150
28/28 [==============================] - 19s 680ms/step - loss: 4.1018 - acc: 0.1066 - val_loss: 3.8659 - val_acc: 0.1307
Epoch 15/150
28/28 [==============================] - 19s 679ms/step - loss: 4.0672 - acc: 0.1105 - val_loss: 3.8585 - val_acc: 0.1304
Epoch 16/150
28/28 [==============================] - 19s 679ms/step - loss: 4.0493 - acc: 0.1119 - val_loss: 3.8428 - val_acc: 0.1350
Epoch 17/150
28/28 [==============================] - 19s 679ms/step - loss: 4.0224 - acc: 0.1167 - val_loss: 3.8412 - val_acc: 0.1340
Epoch 18/150
28/28 [==============================] - 19s 678ms/step

112/112 [==============================] - 22s 200ms/step - loss: 4.3056 - acc: 0.0666 - val_loss: 4.0565 - val_acc: 0.1007
Epoch 21/150
112/112 [==============================] - 22s 201ms/step - loss: 4.2990 - acc: 0.0690 - val_loss: 4.0434 - val_acc: 0.1007
Epoch 22/150
112/112 [==============================] - 23s 202ms/step - loss: 4.2960 - acc: 0.0769 - val_loss: 4.0404 - val_acc: 0.0994
Epoch 23/150
112/112 [==============================] - 22s 201ms/step - loss: 4.2837 - acc: 0.0753 - val_loss: 4.0330 - val_acc: 0.1014
Epoch 24/150
112/112 [==============================] - 22s 201ms/step - loss: 4.2701 - acc: 0.0763 - val_loss: 4.0140 - val_acc: 0.1017
Epoch 25/150
112/112 [==============================] - 23s 201ms/step - loss: 4.2646 - acc: 0.0803 - val_loss: 4.0007 - val_acc: 0.1076
Epoch 26/150
112/112 [==============================] - 22s 201ms/step - loss: 4.2425 - acc: 0.0730 - val_loss: 3.9807 - val_acc: 0.1082
Epoch 27/150
112/112 [==============================] 

Epoch 80/150
112/112 [==============================] - 23s 202ms/step - loss: 3.9543 - acc: 0.1164 - val_loss: 3.7545 - val_acc: 0.1503
Epoch 81/150
112/112 [==============================] - 23s 203ms/step - loss: 3.9514 - acc: 0.1189 - val_loss: 3.7558 - val_acc: 0.1457
Epoch 82/150
112/112 [==============================] - 22s 201ms/step - loss: 3.9579 - acc: 0.1109 - val_loss: 3.7519 - val_acc: 0.1493
Epoch 83/150
112/112 [==============================] - 22s 201ms/step - loss: 3.9753 - acc: 0.1107 - val_loss: 3.7542 - val_acc: 0.1503
Epoch 84/150
112/112 [==============================] - 22s 200ms/step - loss: 3.9510 - acc: 0.1135 - val_loss: 3.7483 - val_acc: 0.1510
Epoch 85/150
112/112 [==============================] - 22s 201ms/step - loss: 3.9533 - acc: 0.1111 - val_loss: 3.7474 - val_acc: 0.1513
Epoch 86/150
112/112 [==============================] - 22s 201ms/step - loss: 3.9508 - acc: 0.1158 - val_loss: 3.7430 - val_acc: 0.1467
Epoch 87/150
112/112 [===================

28/28 [==============================] - 19s 668ms/step - loss: 4.2445 - acc: 0.0907 - val_loss: 3.9825 - val_acc: 0.1246
Epoch 35/150
28/28 [==============================] - 19s 668ms/step - loss: 4.2249 - acc: 0.0926 - val_loss: 3.9781 - val_acc: 0.1285
Epoch 36/150
28/28 [==============================] - 19s 670ms/step - loss: 4.2090 - acc: 0.0975 - val_loss: 3.9743 - val_acc: 0.1278
Epoch 37/150
28/28 [==============================] - 19s 668ms/step - loss: 4.2032 - acc: 0.0959 - val_loss: 3.9666 - val_acc: 0.1285
Epoch 38/150
28/28 [==============================] - 19s 666ms/step - loss: 4.2068 - acc: 0.0952 - val_loss: 3.9741 - val_acc: 0.1210
Epoch 39/150
28/28 [==============================] - 19s 666ms/step - loss: 4.2220 - acc: 0.0920 - val_loss: 3.9615 - val_acc: 0.1252
Epoch 40/150
28/28 [==============================] - 19s 666ms/step - loss: 4.2171 - acc: 0.1006 - val_loss: 3.9535 - val_acc: 0.1246
Epoch 41/150
28/28 [==============================] - 19s 667ms/step

Epoch 28/150
28/28 [==============================] - 22s 795ms/step - loss: 4.0543 - acc: 0.1015 - val_loss: 3.8194 - val_acc: 0.1418
Epoch 29/150
28/28 [==============================] - 21s 738ms/step - loss: 4.0469 - acc: 0.1096 - val_loss: 3.8062 - val_acc: 0.1425
Epoch 30/150
28/28 [==============================] - 20s 724ms/step - loss: 4.0335 - acc: 0.1134 - val_loss: 3.8043 - val_acc: 0.1422
Epoch 31/150
28/28 [==============================] - 20s 717ms/step - loss: 4.0232 - acc: 0.1042 - val_loss: 3.7960 - val_acc: 0.1454
Epoch 32/150
28/28 [==============================] - 22s 800ms/step - loss: 4.0232 - acc: 0.1093 - val_loss: 3.7899 - val_acc: 0.1457
Epoch 33/150
28/28 [==============================] - 20s 729ms/step - loss: 3.9978 - acc: 0.1149 - val_loss: 3.7892 - val_acc: 0.1418
Epoch 34/150
28/28 [==============================] - 19s 685ms/step - loss: 3.9931 - acc: 0.1131 - val_loss: 3.7835 - val_acc: 0.1441
Epoch 35/150
28/28 [==============================] - 2

In [ ]:
X_train, Y_train, X_test, Y_test = data()
val_loss, val_acc = best_model.evaluate(X_test, Y_test);
print("Evalutation of best performing model:")
print("Validation loss: ", val_loss)
print("Validation accuracy: ", val_acc)

Getting data


 83%|████████▎ | 6640/8000 [00:45<00:09, 146.71it/s]

In [20]:
best_model.save(modelPath+modelName);
best_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 90, 90, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 44, 44, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 44, 44, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 44, 44, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [ ]:
# preds = model.predict(x_test, verbose=1)

In [ ]:
# sub = pd.DataFrame(preds)
# # Set column names to those generated by the one-hot encoding earlier
# col_names = one_hot.columns.values
# sub.columns = col_names
# # Insert the column id from the sample_submission at the start of the data frame
# sub.insert(0, 'id', df_test['id'])
# sub.head(10)